In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install keras --upgrade
!pip install zipfile36
!pip install pydicom

     |████████████████████████████████| 2.0 MB 2.6 MB/s 


In [3]:
import keras
import numpy as np
import cv2
import os
import random
import shutil
import pandas as pd
import csv
import pydicom as dicom
import zipfile
from keras import optimizers
from pathlib import Path
from keras.models import Sequential,Model
from keras.layers import Dropout, Flatten, Dense,Input
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.xception import Xception
from keras.callbacks import ModelCheckpoint
from keras.applications.imagenet_utils import preprocess_input
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import RandomNormal
from tensorflow.keras import optimizers

In [4]:
!git clone https://github.com/ieee8023/covid-chestxray-dataset

Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 3641, done.
remote: Total 3641 (delta 0), reused 0 (delta 0), pack-reused 3641
Receiving objects: 100% (3641/3641), 632.96 MiB | 40.62 MiB/s, done.
Resolving deltas: 100% (1450/1450), done.
Checking out files: 100% (1174/1174), done.


In [5]:
archive = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/rsna-pneumonia-detection-challenge.zip') #Extract Kaggle Dataset
for file in archive.namelist():
     archive.extract(file, './All')

In [6]:
fold_num=1 

In [7]:
try:
  os.mkdir('All/All')
except:
  pass

In [8]:
shutil.copy('/content/drive/MyDrive/Colab Notebooks/prepared_csv_files/All.csv','All')
for i in range(1,9): 
  shutil.copy('/content/drive/MyDrive/Colab Notebooks/prepared_csv_files/fold{}/train{}.csv'.format(fold_num,i),'.')
  globals()['train{}'.format(i)]=[]

In [9]:
images=[]
for r,d,f in os.walk('All/stage_2_train_images'):
  for file in f:
    images.append(os.path.join(r,file))
for r,d,f in os.walk('covid-chestxray-dataset/images'):
  for file in f:
   images.append(os.path.join(r,file))

In [10]:
csv_all=pd.read_csv('All/All.csv', nrows=None) #Read the CSV file that contains the names of the images with their labels.
for index, row in csv_all.iterrows(): #This loop reads the images, converts them to suitable format and saves them in the All directory
  if '.png' in row['filename']: #For creating the All.csv we have converted the kaggle dataset images to png format,
                                #but some of the images in the other dataset also are in the format of png, so we use try/except here to distinguish which dataset, the annotation in the CSV file belongs to.
    try:
      png_index=row['filename'].find('.png')
      last_name=row['filename'][:png_index]+'.dcm'
      ds = dicom.dcmread(os.path.join('All/stage_2_train_images',last_name))
      pixel_array_numpy = ds.pixel_array
      imgname = last_name[:-4]+'.png'
      cv2.imwrite(os.path.join('All/All', imgname), pixel_array_numpy)
    except:
      pass
  else:
    img=cv2.imread(os.path.join('covid-chestxray-dataset/images',row['filename']))
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    cv2.imwrite(os.path.join('All/All', row['filename']), gray)  

In [11]:
#optional and can be ignored
All=[] 
all_train=[]
all_test=[]
with open('All/All.csv',newline='', mode='r') as csvfile:
      csvreader = csv.reader(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
      for row in csvreader:
          All.append(row)
for i in range(1,9): 
  with open('train{}.csv'.format(i),newline='', mode='r') as csvfile:
      csvreader = csv.reader(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
      for row in csvreader:
        all_train.append(row)
with open('all_test.csv'.format(i),newline='', mode='w') as csvfile: 
    csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(['filename','class'])
    for row in All:
      if row not in all_train:
        csvwriter.writerow(row)
all_train=[] 
all_test=[]
with open('All/All.csv',newline='', mode='r') as csvfile: 
      csvreader = csv.reader(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
      for row in csvreader:
          All.append(row)
for i in range(1,9): 
  with open('train{}.csv'.format(i),newline='', mode='r') as csvfile:
      csvreader = csv.reader(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
      for row in csvreader:
        all_train.append(row)
with open('all_test.csv'.format(i),newline='', mode='w') as csvfile: 
    csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(['filename','class'])
    for row in All:
      if row not in all_train:
        csvwriter.writerow(row)

In [12]:
for i in range(10): #Shuffle the All list
  random.shuffle(All)
with open('s_test.csv'.format(i),newline='', mode='w') as csvfile: #Create s_test.csv file for evaluating the network during training
    csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(['filename','class'])
    ln=0
    lp=0
    for row in All:
      if row not in all_train:
        if row[1]=='COVID-19':
          csvwriter.writerow(row)
        elif row[1]=='normal':
          if ln<300:
            csvwriter.writerow(row)
            ln+=1
        else:
          if lp<300:
            csvwriter.writerow(row)
            lp+=1

In [13]:

try:
  os.remove('kaggle.zip')
  shutil.rmtree('All/stage_2_train_images')
  shutil.rmtree('All/stage_2_test_images')
  shutil.rmtree('covid-chestxray-dataset')
except:
  pass

In [14]:

train_datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rescale=1./255,zoom_range=0.05,rotation_range=360,width_shift_range=0.05,height_shift_range=0.05,shear_range=0.05)
test_datagen = ImageDataGenerator(rescale=1./255)
test_df = pd.read_csv("s_test.csv")

In [15]:
#training starts
import keras.backend as k
k.clear_session() 
try:
  os.mkdir('models')
except:
  pass
full_name='concatenate'
classes_number=3 
input_tensor=Input(shape=(300,300,3))
######################################################################################################
base_model1 = Xception(weights='imagenet', include_top=False, input_tensor=input_tensor)
features1 = base_model1.output
######################################################################################################
base_model2 = ResNet50V2(weights='imagenet', include_top=False, input_tensor=input_tensor)
features2 = base_model2.output
concatenated=keras.layers.concatenate([features1,features2])
####################################################################################################
conv=keras.layers.Conv2D(1024, (1, 1),padding='same')(concatenated) 
feature = Flatten(name='flatten')(conv)
dp = Dropout(0.5)(feature)
preds = Dense(classes_number, activation='softmax', kernel_initializer=RandomNormal(mean=0.0, stddev=0.001))(dp) 
Concatenated_model = Model(inputs=input_tensor, outputs=preds)
#######################################################
for layer in Concatenated_model.layers:
  layer.trainable = True
Concatenated_model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])
filepath="models/%s-{epoch:02d}-{val_accuracy:.4f}.hdf5"%full_name 
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max') #creating checkpoint 
callbacks_list = [checkpoint]

for epoch_num in range(1,9): 
  train_df =pd.read_csv("train{}.csv".format(epoch_num)) 
  train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory='All/All',
        x_col="filename",
        y_col="class",
        target_size=(300, 300),
        batch_size=20,
        class_mode='categorical',shuffle=True)
  validation_generator = test_datagen.flow_from_dataframe(
          dataframe=test_df,
          directory='All/All',
          x_col="filename",
          y_col="class",
          target_size=(300, 300),
          batch_size=20,
          class_mode='categorical',shuffle=True)
Concatenated_model.fit_generator(train_generator,validation_data = validation_generator, epochs=5,shuffle=True) #start training

94683136/94668760 [==============================] - 1s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 39 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Found 594 validated image filenames belonging to 3 classes.
Found 662 validated image filenames belonging to 3 classes.
Found 594 validated image filenames belonging to 3 classes.
Found 662 validated image filenames belonging to 3 classes.
Found 594 validated image filenames belonging to 3 classes.
Found 662 validated image filenames belonging to 3 classes.
Found 594 validated image filenames belonging to 3 classes.
Found 662 validated image filenames belonging to 3 classes.
Found 594 validated image filenames belonging to 3 classes.
Found 662 validated image filenames belonging to 3 classes.
Found 594 validated image filenames belonging to 3 classes.
Found 662 validated image filenames belonging to 3 classes.
Found 594 validated image filenames belonging to 3 classes.
Found 662 validated image filenames belonging to 3 classes.
Found 594 validated image filenames belonging to 3 classes.
Found 662 validated image filenames belonging to 3 classes.
Epoch 1/5
30/30 [=======================

In [16]:
#saving model
import pickle
with open('saved-model','wb') as f:
  pickle.dump(Concatenated_model,f)

# load model
# with open('saved-model','rb') as f:
#   loaded_model = pickle.load(f)

INFO:tensorflow:Assets written to: ram://e7f0136c-d6a6-411a-a6e1-e2f34a4a6270/assets


In [17]:
trained_models=[]
for r,d,f in os.walk('models'):
  for file in f:
    trained_models.append(os.path.join(r,file)) 
reports={}
test_df = pd.read_csv("all_test.csv") #Load the full test CSV file that includes 11302 images
validation_generator = test_datagen.flow_from_dataframe( 
        dataframe=test_df,
        directory='All/All',
        x_col="filename",
        y_col="class",
        target_size=(300, 300),
        batch_size=20,
        class_mode='categorical',shuffle=True)
for trained_model in trained_models:
  k.clear_session()
  net=keras.models.load_model(trained_model) #load model
  covid_label= validation_generator.class_indices['COVID-19'] 
  pneu_label= validation_generator.class_indices['pneumonia']  
  normal_label= validation_generator.class_indices['normal']  
  tp=0 #True Positives
  fp=0 #False Positives
  anum=0 #All the images numbers
  ###########
  wrong_covid=0
  correct_covid=0
  not_detected_covid=0
  covid_num=0
  ###########
  wrong_pneu=0
  correct_pneu=0
  not_detected_pneu=0
  pneu_num=0
  ############
  wrong_normal=0
  correct_normal=0
  not_detected_normal=0
  normal_num=0
  ##############
  wrong_covid_normal=0 
  wrong_covid_pneu=0   
  wrong_pneu_covid=0   
  wrong_pneu_normal=0 
  wrong_normal_pneu=0  
  wrong_normal_covid=0  
  ################
  for num,img_name in enumerate(validation_generator.filenames): #load image
    gt_ind=validation_generator.classes[num] #get the loaded image class index
    img=cv2.resize(cv2.imread(os.path.join('All','All',img_name)),(300,300)) #resize image
    img=img.astype('float32') / 255.0 #scale the image
    pred_ind=np.argmax(net.predict(np.expand_dims(img,axis=0))[0]) #get the predicted class index

    anum+=1 #count the number of images


    if gt_ind==covid_label:
      covid_num+=1
    if gt_ind==pneu_label:
      pneu_num+=1
    if gt_ind==normal_label:
      normal_num+=1
    ##################
    if gt_ind==covid_label and pred_ind==covid_label: 
      correct_covid+=1
    if gt_ind==covid_label and pred_ind!=covid_label:
      not_detected_covid+=1
      if pred_ind==pneu_label:
        wrong_covid_pneu+=1
      elif pred_ind==normal_label:
        wrong_covid_normal+=1
    if gt_ind!=covid_label and pred_ind==covid_label:
      wrong_covid+=1
    ###########################################
    if gt_ind==normal_label and pred_ind==normal_label: 
      correct_normal+=1
    if gt_ind==normal_label and pred_ind!=normal_label:
      not_detected_normal+=1
      if pred_ind==pneu_label:
        wrong_normal_pneu+=1
      elif pred_ind==covid_label:
        wrong_normal_covid+=1
    if gt_ind!=normal_label and pred_ind==normal_label:
      wrong_normal+=1
    ###########################################
    if gt_ind==pneu_label and pred_ind==pneu_label: 
      correct_pneu+=1
    if gt_ind==pneu_label and pred_ind!=pneu_label:
      not_detected_pneu+=1
      if pred_ind==normal_label:
        wrong_pneu_normal+=1
      elif pred_ind==covid_label:
        wrong_pneu_covid+=1
    if gt_ind!=pneu_label and pred_ind==pneu_label:
      wrong_pneu+=1
    ###########################################

    if pred_ind==gt_ind:
      tp+=1
    else:
      fp+=1

Found 22604 validated image filenames belonging to 3 classes.
